### View f&uuml;r Spiel "Steine setzen und verschieben"
Zur Darstellung des Spielzustandes benutzen wir eine
MultiCanvas mit 2 Layern (Hintergrund, Vordergrund).



Mit `view = View(game)` wird eine View f&uuml;r die Spielinstanz `game` erstellt.  
Die `__init__` Methode weist dem Attribut `game.callback` die Funktion  
`lambda event, data: self.callback(even, data)` zu.  

In [ ]:
from ipycanvas import MultiCanvas
class View:
    
    pts = [(180, 100), (140, 30), (60, 30), (20, 100),  (60, 170),  (140, 170)]
    colors = ['LightSalmon', 'Salmon', 'Red','Crimson',  'FireBrick', 'DarkRed']
    r = 10
   
    def __init__(self, game):
        self.game = game
        # callback registrieren
        game.callback = lambda event, data: self.callback(event, data)
        
        self.mcanvas = MultiCanvas(2, width = 200, height = 200, 
                                   layout = {'border': '1px solid black'}
                                  )
        self.bg, self.fg = self.mcanvas
       
        for pt in self.pts:
            self.bg.fill_circle(*pt, 3)
            
    def place_stone(self, idx, color='black'):
        x, y = self.idx2pos(idx)
        self.fg.fill_style = color
        self.fg.fill_circle(x, y, self.r)
        
    def remove_stone(self, idx):
        x, y = self.idx2pos(idx)
        self.fg.clear_rect(x-self.r-0.5, y-self.r-0.5, 2*self.r+1)
        
    def move_stone(self, src, target, color='black'):
        self.remove_stone(src)
        self.place_stone(target, color)
        
    def idx2pos(self, idx):
        return self.pts[idx]
    
    def callback(self, move_type, idxs):
        if move_type == 'place_stone':
            src = idxs[0]
            self.place_stone(src, self.colors[src])
        if move_type == 'move_stone':
            src, target = idxs
            self.move_stone(src, target, self.colors[target])
            
    def _ipython_display_(self):
        display(self.mcanvas)

In [ ]:
# Klasse Game und Funktion engine importieren
from game_setzen_verschieben import Game

In [ ]:
game = Game()
view = View(game)
view

In [ ]:
# Teste ob View auf Callback reagiert
game.place(0)
game

In [ ]:
game.move(0, 1)

In [ ]:
game.move(1, 2)

### Steine mit der Maus setzen und bewegen
- `on_mouse_down(x, y)`: Finde mit `idx = get_closest(self.pts, (x, y))` den Index des  n&auml;chsten Punktes in `pts`. Ist `idx` ungleich `None`, speichere ihn in `fg.selectet_point` und setze einen Stein auf diesen Punkt.
- `on_mouse_up(x, y)`: Ist `idx = get_closest(self.pts, (x, y))` ungleich `None`,
verschieben den Stein von Ecke `fg.selectet_point` auf Ecke `idx`.

In [ ]:
from get_closest import get_closest
from ipywidgets import Output
err_msg = Output(layout = {'border': '1px solid black'})
from ipycanvas import MultiCanvas

class View:
    pts = [(180, 100), (140, 30), (60, 30), (20, 100),  (60, 170),  (140, 170)]
    colors = ['LightSalmon', 'Salmon', 'Red','Crimson',  'FireBrick', 'DarkRed']
    r = 10
    
    def __init__(self, game):
        self.game = game
        # callback registrieren
        game.callback = lambda event, data: self.callback(event, data)
        
        self.mcanvas = MultiCanvas(2, width = 200, height = 200, 
                                   layout = {'border': '1px solid black'}
                                  )
        self.bg, self.fg = self.mcanvas
        for pt in self.pts:
            self.bg.fill_circle(*pt, 3)
            
        # Attribut fuer selektiertes Feld, Callbacks fuer Maus-Events registrieren
        self.selected_field = None
        self.mcanvas.on_mouse_down(self.on_mouse_down)
        self.mcanvas.on_mouse_up(self.on_mouse_up)
       
    def place_stone(self, idx, color='black'):
        x, y = self.pts[idx]
        self.fg.fill_style = color
        self.fg.fill_circle(x, y, self.r)
        
    def remove_stone(self, idx):
        x, y = self.pts[idx]
        self.fg.clear_rect(x-self.r-0.5, y-self.r-0.5, 2*self.r+1)
        
    def move_stone(self, src, target, color='black'):
        self.remove_stone(src)
        self.place_stone(target, color)
        
    def callback(self, move_type, idxs):
        if move_type == 'place_stone':
            src = idxs[0]
            self.place_stone(src, self.colors[src])
        if move_type == 'move_stone':
            src, target = idxs
            self.move_stone(src, target, self.colors[target])
            
    @err_msg.capture()        
    def on_mouse_down(self, x, y):
        idx = get_closest(self.pts, (x, y))
        if idx is not None:
            self.selected_field = idx
            # Zug an Game-Instanz weitergeben
            self.game.place(idx) 
        
    @err_msg.capture()   
    def on_mouse_up(self, x, y):   
        if self.selected_field is None:
            return
        target = get_closest(self.pts, (x, y))
        if target is not None:
            # Zug an Game-Instanz weitergeben
            self.game.move(self.selected_field, target)  
            self.selected_field = None       
            
    def _ipython_display_(self):
        display(self.mcanvas, err_msg)

In [ ]:
game = Game()
view = View(game)
view